Fuente de los datos dataset Bacterias:

https://www.kaggle.com/datasets/nwheeler443/gono-unitigs/data

In [238]:
# Procesado de los dataset y mapas folium
import folium
import geopandas as gpd
from branca.colormap import linear
from datetime import datetime
from folium.plugins import TimeSliderChoropleth
from datetime import datetime
import numpy as np
from branca.colormap import linear
import pandas as pd
import math

In [239]:
df_bacterias = pd.read_csv('datos/dataset_procesado_bacterias_resistentes.csv')

In [240]:
df_bacterias.head()

,Unnamed: 0,Sample_ID,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
0,0,ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
1,1,ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
2,2,ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
3,3,ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
4,4,ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


Transformamos los años en Fechas

In [241]:
def correct_date(date_str):
    list_dates = date_str.split("-")
    year = list_dates[0]
    month = list_dates[1]
    day = list_dates[2]
    
    if len(day) == 1:
        day = "0" + day
    if len(month) == 1:
        month = "0" + month
        
    return "/".join([day, month, year])

def correct_date2(year_float):
    if(math.isnan(year_float) == False):
        year = str(year_float)[:4]
    else:
        year='2000'
    day = '01'
    month = '01'
    
    return "/".join([day, month, year])

In [242]:
df_bacterias["NEW_DATE"] = df_bacterias["Year"].apply(correct_date2)
df_bacterias.drop(['Unnamed: 0'], inplace=True, axis=1)
df_bacterias["DATE"] = pd.to_datetime(df_bacterias["NEW_DATE"], format='%d/%m/%Y')
df_bacterias=df_bacterias.drop(['NEW_DATE'], axis=1)
df_bacterias=df_bacterias.drop(['Year'], axis=1)
df_bacterias.head()

,Sample_ID,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr,DATE
0,ERR1549286,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
1,ERR1549290,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
2,ERR1549291,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
3,ERR1549287,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
4,ERR1549288,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01


In [243]:
df_bacterias["DATE_for_Folium"] = (df_bacterias["DATE"].astype('int64')// 10**9).astype('U10')
# df_bacterias.drop(['DATE'], axis=1, inplace=True)
df_bacterias['Date'] = df_bacterias['DATE_for_Folium']
df_bacterias.drop(['DATE_for_Folium'], axis=1, inplace=True)
df_bacterias.head()

,Sample_ID,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr,DATE,Date
0,ERR1549286,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01,1420070400
1,ERR1549290,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01,1420070400
2,ERR1549291,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01,1420070400
3,ERR1549287,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01,1420070400
4,ERR1549288,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01,1420070400


In [244]:
df_fecha_DATE = df_bacterias[['Date', 'DATE']].drop_duplicates()
df_fecha_DATE.rename(columns={
    'DATE': 'Fecha'
}, inplace=True)
df_fecha_DATE.head()

,Date,Fecha
0,1420070400,2015-01-01
14,820454400,1996-01-01
15,852076800,1997-01-01
16,883612800,1998-01-01
17,946684800,2000-01-01


In [295]:
df_bacteria_groupby_conutry = df_bacterias.groupby(['Country', 'Date'],
                                                   as_index=False).count()[[
    'Country', 'Date', 'Sample_ID']].rename(columns={
    'Sample_ID': 'Cuenta',
    'Country': 'Country',    
})

In [296]:
df_bacteria_groupby_conutry

,Country,Date,Cuenta
0,Angola,915148800,1
1,Argentina,1199145600,1
2,Armenia,1199145600,1
3,Australia,1072915200,6
4,Australia,694224000,1
5,Australia,820454400,2
6,Australia,852076800,1
7,Australia,978307200,1
8,Austria,1262304000,2
9,Austria,1356998400,54


In [297]:
df_bacteria_groupby_conutry['Date'] = df_bacteria_groupby_conutry['Date'].astype('int64')
df_fecha_DATE['Date'] = df_fecha_DATE['Date'].astype('int64')
df_fecha_DATE.dtypes

Date              int64
Fecha    datetime64[ns]
dtype: object

In [298]:
df_bacteria_groupby_conutry.dtypes

Country    object
Date        int64
Cuenta      int64
dtype: object

In [299]:
df_bacteria_groupby_conutry_for_humans = df_bacteria_groupby_conutry.join(df_fecha_DATE.set_index('Date'),
                                                                          how='left', on='Date',
                                                                          rsuffix='_right')

In [300]:
df_bacteria_groupby_conutry.head()

,Country,Date,Cuenta
0,Angola,915148800,1
1,Argentina,1199145600,1
2,Armenia,1199145600,1
3,Australia,1072915200,6
4,Australia,694224000,1


In [301]:
df_fecha_DATE.head()

,Date,Fecha
0,1420070400,2015-01-01
14,820454400,1996-01-01
15,852076800,1997-01-01
16,883612800,1998-01-01
17,946684800,2000-01-01


In [302]:
df_bacteria_groupby_conutry_for_humans.head()

,Country,Date,Cuenta,Fecha
0,Angola,915148800,1,1999-01-01
1,Argentina,1199145600,1,2008-01-01
2,Armenia,1199145600,1,2008-01-01
3,Australia,1072915200,6,2004-01-01
4,Australia,694224000,1,1992-01-01


Ya podemos visualizar el dataset en un Mapa Folium

In [303]:
# Cargar el archivo de datos geoespaciales (GeoJSON) para los países
world_geojson_path = 'datos/ne_50m_admin_0_countries.geojson'
gdf = gpd.read_file(world_geojson_path)

# m = folium.Map(location=(30, 10), zoom_start=3, tiles="cartodb positron")

In [304]:
n_periods = len(np.unique(df_bacteria_groupby_conutry['Date']))
datetime_index = pd.date_range("2016-1-1", periods=n_periods, freq="M")
dt_index_epochs = datetime_index.astype("int64") // 10 ** 9
dt_index = dt_index_epochs.astype("U10")

styledata = {}

for country in gdf.index:    
    df = pd.DataFrame(
        {
            "color": np.random.normal(size=n_periods),
            "opacity": np.random.normal(size=n_periods),
        },
        index=dt_index,
    )
    df = df.cumsum()
    # df.sample(n_sample, replace=False).sort_index()
    styledata[country] = df

styledata

{0:                color   opacity
 1454198400 -1.506081  1.511274
 1456704000 -1.748727  0.033444
 1459382400 -2.645617 -0.602472
 1461974400 -0.122576  0.219786
 1464652800 -0.974712  0.637347
 1467244800 -2.273058  1.177362
 1469923200 -3.580094 -0.542786
 1472601600 -4.358164 -2.162125
 1475193600 -3.465399 -1.386790
 1477872000 -2.858359 -1.487920
 1480464000 -2.307612 -1.100166
 1483142400 -1.722669 -0.076571
 1485820800  0.537199 -1.092139
 1488240000 -0.893675 -1.082625
 1490918400 -0.711352 -2.091876
 1493510400 -1.243719 -1.498525
 1496188800 -0.311694 -1.857107
 1498780800  0.240826 -1.909534
 1501459200  0.866700 -2.903461
 1504137600  2.520024 -4.611407
 1506729600  1.924059 -5.049153
 1509408000  0.857970 -4.118012
 1512000000  1.051619 -3.267076
 1514678400 -0.088312 -2.146671
 1517356800 -0.199145 -3.043627
 1519776000  1.951845 -2.039492
 1522454400  1.986450 -2.610541
 1525046400  0.922375 -3.645298
 1527724800  0.616597 -3.988119
 1530316800  1.468643 -4.439637,
 1: 

In [305]:
max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data["color"].max())
    min_color = min(max_color, data["color"].min())
    max_opacity = max(max_color, data["opacity"].max())
    max_opacity = min(max_color, data["opacity"].max())

In [306]:
from branca.colormap import linear

cmap = linear.PuRd_09.scale(min_color, max_color)


def norm(x):
    return (x - x.min()) / (x.max() - x.min())


for country, data in styledata.items():
    data["color"] = data["color"].apply(cmap)
    data["opacity"] = norm(data["opacity"])
    
# print(data)

In [307]:
ids = set(styledata.get(1).head().index)
l_ids = list(ids)

In [308]:
cmap = linear.PuRd_09.scale(min_color, max_color)

In [309]:
styledict = {
    str(country): data.to_dict(orient="index") for country, data in styledata.items()
}

In [310]:
# gdf.columns.values

In [312]:
# Definimos el mapa
folium.Choropleth(
    geo_data=gdf,
    data=df_bacteria_groupby_conutry,
    columns=["Country", "Cuenta"],
    key_on="feature.id",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Numero de Bacterias resistentes por Pais",    
).add_to(m)

# Crear un control deslizante de tiempo
time_slider = TimeSliderChoropleth(
    data=gdf.to_json(),        
    styledict=styledict,
    name='Time Slider').add_to(m)

In [1]:
m

NameError: name 'm' is not defined

In [120]:
m.save('html/mapa_mundial_bacterias_resistentes.html')

In [127]:
pd.set_option('display.max_rows', None)
df_bacteria_groupby_conutry.sort_values(['Date', 'Cuenta'],
                                        axis = 0, ascending = False,
                                        na_position = "first")

,SOVEREIGNT,Date,Cuenta
221,USA,978307200,28
43,Canada,978307200,10
113,Japan,978307200,8
7,Australia,978307200,1
205,UK,946684800,249
220,USA,946684800,37
42,Canada,946684800,8
112,Japan,946684800,6
41,Canada,915148800,8
174,Thailand,915148800,4


In [216]:
df_bacteria_groupby_conutry_for_humans[df_bacteria_groupby_conutry_for_humans['Cuenta'] == max(df_bacteria_groupby_conutry['Cuenta'])]

,SOVEREIGNT,Date,Cuenta,Fecha
121,New_Zealand,1420070400,398,2015-01-01


In [218]:
df_bacteria_groupby_conutry_for_humans[df_bacteria_groupby_conutry_for_humans['Date'] == min(df_bacteria_groupby_conutry['Date'])]

,SOVEREIGNT,Date,Cuenta,Fecha
172,Thailand,283996800,1,1979-01-01


In [219]:
df_bacteria_groupby_conutry_for_humans.nlargest(10, 'Cuenta')

,SOVEREIGNT,Date,Cuenta,Fecha
121,New_Zealand,1420070400,398,2015-01-01
205,UK,946684800,249,2000-01-01
215,USA,1293840000,176,2011-01-01
213,USA,1230768000,146,2009-01-01
214,USA,1262304000,143,2010-01-01
216,USA,1325376000,128,2012-01-01
163,Spain,1356998400,116,2013-01-01
141,Portugal,1356998400,108,2013-01-01
194,UK,1356998400,108,2013-01-01
217,USA,1356998400,99,2013-01-01


In [ ]:
#----------------------------------------------------------------------------------
data_to_color = "TOTAL_DEATHS"
cats, bins =  pd.qcut(df_covid[data_to_color].unique()[np.argsort(df_covid[data_to_color].unique())], q = 9, retbins = True)
cats = cats.unique()

#----------------------------------------------------------------------------------

# value we will iterate in order to create the styledict
ccaas = list(df_covid["id"].unique())
dates = list(df_covid["DATE_for_Folium"].unique())

# create the color dict and color column
df_covid["COLORS"] = df_covid[data_to_color].apply(get_hex_colors_2, args = [cats]) # we create a colum in the df so that we can iterate and create the styledict

# creates the styledict for the map
styledict = {}

# iterate the populate the styledict
for ccaa in ccaas:
    styledict[str(ccaa)] = {date: {'color': df_covid[(df_covid["id"] == ccaa) & (df_covid["DATE_for_Folium"] == date)]["COLORS"].values[0],
                                   'opacity': 0.6} for date in dates}
    
# creates and renders the Folium map
m = folium.Map(location=(40, 0), tiles='OpenStreetMap', zoom_start=6)

g = TimeSliderChoropleth(
    gdf.set_index("id").to_json(), # get's the coordinates for each id 
    styledict = styledict # styledict contains for each id the timestamp and the color to plot.
)

m.add_child(g)

#--------------------------------------------------------------------------------------
# Let's create a legend for folium
# https://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd

from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <style>html, body {overflow-y: hidden;}</style>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FFFFFF;opacity:0.6;'></span>No cases</li>
    <li><span style='background:#fff5f0;opacity:0.6;'></span>1 Quantile</li>
    <li><span style='background:#fee0d2;opacity:0.6;'></span>2 Quantile</li>
    <li><span style='background:#fcbba1;opacity:0.6;'></span>3 Quantile</li>
    <li><span style='background:#fc9272;opacity:0.6;'></span>4 Quantile</li>
    <li><span style='background:#fb6a4a;opacity:0.6;'></span>5 Quantile</li>
    <li><span style='background:#ef3b2c;opacity:0.6;'></span>6 Quantile</li>
    <li><span style='background:#cb181d;opacity:0.6;'></span>7 Quantile</li>
    <li><span style='background:#a50f15;opacity:0.6;'></span>8 Quantile</li>
    <li><span style='background:#67000d;opacity:0.6;'></span>9 Quantile</li>
    <li><span style='background:#000000;opacity:0.6;'></span>Other</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)